# Get Setup

In [1]:
# we need torch 1.12+ and torchvision 0.13+
%load_ext autoreload
%autoreload 2
try:
    print("OK")
    import torch
    import torchvision

    assert float(torch.__version__.split(".")[0]) > 1 or int(
        torch.__version__.split(".")[1]) > 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U --pre torch torchvision torchaudio -- extra-index-url https: // download.pytorch.org/whl/nightly/cu113
    import torch
    import torchvision

    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

OK
torch version: 2.0.1+cu117
torchvision version: 0.15.2+cu117


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms

# try to get torchinfo, if doesn't exist, install it
try:
    from torchinfo import summary
except:
    !pip install -q torchinfo
    from torchinfo import summary

# try to import the going_modular directory, download it from github if it doesn't exist
try:
    from going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    raise FileNotFoundError

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Get Data

In [4]:
image_path = download_data(
    source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
    destination="pizza_steak_sushi")
image_path

[INFO] data\pizza_steak_sushi directory exists, skipping download.


WindowsPath('data/pizza_steak_sushi')

In [5]:
train_dir = image_path / "train"
test_dir = image_path / "test"
train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

# creat dataloaders

In [6]:
from going_modular import data_setup
from torchvision import transforms

# creat image size (224,224)
IMG_SIZE = 224  # comes from table 3 of the ViT paper

# create transforms
manual_transforms = transforms.Compose([
    transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

print(f"manually created transforms: {manual_transforms}")

manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)


In [12]:
# create batch size 32, the paper used 4096.
BATCH_SIZE = 32
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    test_transform=manual_transforms,
    batch_size=BATCH_SIZE, 
    num_workers=0)

len(train_dataloader), len(test_dataloader), class_names  # (32, 32, 3a)

(8, 3, ['pizza', 'steak', 'sushi'])

In [14]:
# get batch images
image_batch, label_batch = next(iter(train_dataloader))

image, label = image_batch[0], label_batch[0]
image.shape, label

(torch.Size([3, 224, 224]), tensor(0))